In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
from sklearn.impute import KNNImputer
from sklearn.metrics import recall_score, precision_score 
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate 
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from category_encoders.count import CountEncoder
from training_functions import get_data, get_feature_names_order

# 0. CARGA DE DATAS.

In [2]:
path='../outputs/'
df_train = get_data(name_sav='wA_train.sav', path=path)
df_test = get_data(name_sav='wA_test.sav', path=path)

# 1. CONTEXTO DE LA MINICOMPETENCIA.

Estamos ante un problema de clasificación binaria con un evento de desbalanceo importante. 

Se hará una minicompetencia de forma arbitraria, en el sentido de que solo habrá dos modelos a considerar, uno "clasico" y uno muy sofisticado. Estos son `RandomForestClassifier` e `HistGradientBoostingClassifier`.

Los puntos generales a considerar en esta competencia son:
* Se usarán una semilla arbitraria (`seed = 5000`).

* Ambos modelos se entrenarán con los valores de sus hiperpárametros a como están por default en sus configuraciones desde `scikit-learn`. 


* El mejor modelo será el que tenga un mejor performance en la métrica __f1__ de __cross-validation__, porque, al combinar métricas como el __recall__ y el __precision__, está métrica da una idea del buen rendimiento general en ambos conjuntos, aunque estos estén desequilibrados.
De hecho, el __f1__ puede interpretarse como una especie de correlación, y entre más cercano a 1, se tendrá tanto un alto __precision__ como un alto __recall__, lo que implica una buena capacidad para clasificar correctamente las muestras positivas y minimizar los falsos positivos y falsos negativos.

* Aunque el ganador lo dicidirá el __f1__, también se calculará la matriz de confusión de __testeo__, así como los scores de __precision__, __recall__, __coeficiente de correlación de Matthews (m_c)__, de __testeo__ y __cross-validation__, esto con fines informativos. Sin embargo, también pueden ayudar a elegir al modelo ganador en caso de empate.

* Existen varios métodos para lidiar con el desbalanceo, entre submuestreos de la clase mayoritaria o sobremuestreos de la clase minoritaria. Aunque un método muy famoso de sobremuestreo es el `SMOTE`, en esta prueba se usará el método de submuestreo `RandomUnderSampler`. Esto es debido a que simplemente, por experiencia, se ha tenido mayor éxito en el pasado usando métodos de submuestreo.

* El subuestreo se hará de tal forma que se tenga un equilibrio total entre la clase minoritaria y la mayoritaria. Esto equivale a tener una estrategia de muestreo igual a 1 (`ratio_balance = 1`).

* Para la codificación de variables categóricas, no se hará uso de ténicas clásicas como el `OneHotEncoder` o el`LabelEncoder`,  si no del `CountEncoder`, esto para evitar la complejización del modelo. El `CountEncoder`, en su versión normalizada, consiste en el porcentaje de representación que tiene cierta clase en una variable categórica.

* No se escalarán las variables numéricas, esto debido a que la mayoría de ellas se mueven en un rango similar de valores.

* Se hará uso de __Pipilines__, es decir, de tuberias de proprocesamiento que contendrán pasos de transformación y de estimación necesarios.

* El uso de __Pipilines__ será especialemnte importante sobretodo en la etapa de __cross-validation k-folds__, para garatizar la repetición de los procesos de imputación, codificación, submuestreo y ajuste del modelo, en cada iteración-__fold__.

* Solo usaremos una cantidad pequeña de __folds__ (`k_folds = 4`).

* El proceso de __cross-validatión__ contempla una estrategia de división estratificada (`StratifiedKFold`). Esto con el fin de que en cada iteración-__fold__ la variable objetivo tenga aproximadamente la misma configuración de porcentajes de clase a como se tiene en el conjunto de __Entrenamiento__. De tal suerte que el proceso de submuetreo será aplicado en cada __fold__ y esto será posible con ayuda los __Pipilines__ ofrecidos por la librería `imbalanced-learn`.

# 2. CONFIGURACIONES GENERALES.

Se establecen los siguientes valores por default:

In [3]:
seed = 5000
ratio_balance = 1
k_folds = 4
verbose = 10

Se establece el listado de features y la variable objetivo.

In [4]:
features_names =  ['Location',
                   'MinTemp',
                   'Rainfall',
                   'WindGustDir',
                   'WindGustSpeed',
                   'WindDir9am',
                   'WindDir3pm',
                   'WindSpeed3pm',
                   'Humidity9am',
                   'Humidity3pm', 
                   'Pressure9am',
                   'RainToday', 
                   'month',
                   'week_of_year']
objective_name = 'RainTomorrow'

Se obtiene el dataset de features en el conjunto de __Entrenamiento__.

In [5]:
original_data = df_train.copy()
data_features = original_data[features_names]

Se obtienen los nombres de variables numéricas y de las categóricas.

In [6]:
float_names = list(data_features.select_dtypes(include='float64').columns)
categorical_names = list(data_features.select_dtypes(include='object').columns)

Listado de scores de interés:

In [7]:
scores = {'f1': 'f1',
          'precision': 'precision',
          'recall': 'recall',
          'm_c': make_scorer(matthews_corrcoef)}

Lo siguiente es el __Pipiline__ correspondiente al preprocesamiento de variables numéricas. Por ahora, solo contiene el imputador no parámetrico `KNNImputer` que solo se aplicará cuando se haga uso del estimador `RandomForestClassifier`. Lo anterior debido a que el estimador `HistGradientBoostingClassifier` ya contempla la presencia de missing en su ejecución.

In [8]:
numeric_transformer = Pipeline(steps=[('imputer', KNNImputer(n_neighbors=3, weights="uniform"))])

El siguiente __Pipilene__ corresponde al procesamiento de variables categóricas. Contiene el codificación `CountEncoder`normalizada.

In [9]:
categorical_transformer = Pipeline(steps=[('CountEncoder', CountEncoder(normalize=True))])

Por último, debido al uso de tranformaciones que afectan ya sea solo a variables categóricas o solo variables numéricas, el orden en que aparecen estos grupos de variables en los datasets es importante. Por otro lado, las tranformaciones que se aplican también dependen del estimador que se esté usando, por ejemplo, el `HistGradientBoostingClassifier` no requiere una imputación de valores previa.

El orden en que van aplicandose la tranformaciones va corriendo a la derecha del dataset al grupo de variables al que se termina de aplicar dichas tranformaciones (ver seccion *Notes* [aquí.](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html))

Para lidiar con este problema se hace uso de la función __get_feature_names_oder__ (ver información de ella en el script __training_functions.py__). 

# 3. ENTRENAMIENTO DE HISTOGRAM GRADIENT BOOSTING.

Entrenemos el estimador `HistGradientBoostingClassifier`.

In [10]:
model_name = 'HistGradientBoostingClassifier'

In [11]:
# orden de las variables, depende del estimador escogido.
feature_names_order = get_feature_names_order(model_name=model_name,
                                              float_names=float_names,
                                              categorical_names=categorical_names)

Se obtienen las versiones definitivas de los conjuntos de __Entrenamiento__ y de __Testeo__.

In [12]:
X_train= data_features[feature_names_order]
y_train = original_data[objective_name]
X_test= df_test[feature_names_order]
y_test = df_test[objective_name]

La __objeto (clase)__ de preprocesamiento solo contemplará la codificación de variables categóricas.

In [13]:
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[('categorical', categorical_transformer, categorical_names)])

Se muestra el orden en que se aplicarán los procesos de tranformación en el __Pipiline general__:
1. Preprocesamiento de variables (codificación).
2. Submuestro.
3. Ajuste de modelo.

In [14]:
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  HistGradientBoostingClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['Location', 'WindGustDir',
                                                   'WindDir9am', 'WindDir3pm',
                                                   'RainToday', 'month',
                                                   'week_of_year'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator',
                 HistGradientBoostingClassifier(random_state=5000))])

## 3. 1 Ajuste del modelo para obtener métricas de testeo.

In [15]:
clf = transform.fit(X_train, y_train)

In [16]:
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [17]:
f_1_testeo = f1_score(y_test, y_pred)
precision_testeo = precision_score(y_test, y_pred)
recall_testeo = recall_score(y_test, y_pred)
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [18]:
print('f1-testeo:', round(f_1_testeo, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.63
precision-testeo: 0.53
recall-testo: 0.77
m_c-testeo: 0.5
matriz de confusión de testeo [[15942  4085]
 [ 1375  4571]]


## 3.2 Cross-validation.

In [19]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)

In [20]:
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.633) m_c: (test=0.517) precision: (test=0.540) recall: (test=0.764) total time=   2.1s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    2.1s


[CV] END  f1: (test=0.634) m_c: (test=0.519) precision: (test=0.533) recall: (test=0.782) total time=   2.0s
[CV] START .....................................................................
[CV] END  f1: (test=0.633) m_c: (test=0.518) precision: (test=0.531) recall: (test=0.784) total time=   2.1s
[CV] START .....................................................................
[CV] END  f1: (test=0.636) m_c: (test=0.522) precision: (test=0.536) recall: (test=0.780) total time=   2.4s


[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    8.7s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    8.7s


Se obtienen las métricas de __cross-validation__.

In [21]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.63
precision-cv: 0.53
recall-cv: 0.78
m_c-cv: 0.52


# 4. ENTRENAMIENTO DE RANDOM FOREST.

__¡¡¡ ADVERTENCIA !!!:__ El entrenamiento y __cross-validation__ de este modelo puede durar mucho tiempo debido al proceso de imputación de datos que lleva a cabo.

Entrenemos el estimador `RandomForestClassifier`.

In [22]:
model_name = 'RandomForestClassifier'

In [23]:
# orden de las variables, depende del estimador escogido.
feature_names_order = get_feature_names_order(model_name=model_name,
                                              float_names=float_names,
                                              categorical_names=categorical_names)

Se obtienen las versiones definitivas de los conjuntos de __Entrenamiento__ y de __Testeo__.

In [24]:
X_train= data_features[feature_names_order]
y_train = original_data[objective_name]
X_test= df_test[feature_names_order]
y_test = df_test[objective_name]

La __objeto (clase)__ de preprocesamiento contemplará tanto la imputación de valores en variables numéricas como la codificación de variables categóricas.

In [25]:
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[('numeric', numeric_transformer, float_names),
                                               ('categorical', categorical_transformer, categorical_names)])

Se muestra el orden en que se aplicarán los procesos de tranformación en el __Pipiline general__:
1. Preprocesamiento de variables (imputación y codificación).
2. Submuestro.
3. Ajuste de modelo.

In [26]:
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  RandomForestClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3))]),
                                                  ['MinTemp', 'Rainfall',
                                                   'WindGustSpeed',
                                                   'WindSpeed3pm',
                                                   'Humidity9am', 'Humidity3pm',
                                                   'Pressure9am']),
                                                 ('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['Location', 'WindGustDir',
                                                   'WindDir9am', 'WindDir3pm',
                                                   'RainToday', 'month',
                                                   'week_of_year'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator', RandomForestClassifier(random_state=5000))])

## 4. 1 Ajuste del modelo para obtener métricas de testeo.

In [27]:
clf = transform.fit(X_train, y_train)

In [28]:
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [29]:
f_1_testeo = f1_score(y_test, y_pred)
precision_testeo = precision_score(y_test, y_pred)
recall_testeo = recall_score(y_test, y_pred)
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [30]:
print('f1-testeo:', round(f_1_testeo, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.62
precision-testeo: 0.52
recall-testo: 0.76
m_c-testeo: 0.49
matriz de confusión de testeo [[15841  4186]
 [ 1438  4508]]


## 4.2 Cross-validation.

In [31]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)

In [32]:
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.626) m_c: (test=0.507) precision: (test=0.530) recall: (test=0.764) total time= 2.8min
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:  2.8min


[CV] END  f1: (test=0.622) m_c: (test=0.502) precision: (test=0.523) recall: (test=0.767) total time= 2.8min
[CV] START .....................................................................
[CV] END  f1: (test=0.629) m_c: (test=0.513) precision: (test=0.528) recall: (test=0.778) total time= 2.9min
[CV] START .....................................................................
[CV] END  f1: (test=0.628) m_c: (test=0.511) precision: (test=0.530) recall: (test=0.771) total time= 2.9min


[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed: 11.3min
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed: 11.3min


Se obtienen las métricas de __cross-validation__.

In [33]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.63
precision-cv: 0.53
recall-cv: 0.77
m_c-cv: 0.51


# 5. MODELO GANADOR.

Hay un empate entre los modelos cuando se contemplan los scores __f1__ y __precision__ de __cross-validation__ (0.63 y 0.53). 

El desempate se da al observar una mejora en los scores de __recall__ y __m_c__ de __cross-validation__ en el estimador 
`HistGradientBoostingClassifier` frente a los del `RandomForestClassifier` (0.78 y 0.52 vs 0.77 y 0.51). Además, los scores de __testeo__ del primero superan a los scores de __testeo__ del segundo.

Siendo así, se elige al `HistGradientBoostingClassifier` por tener un mejor performance en las métricas de __cross-validation__ y de __testeo__. Además, es más rápido al considerar desde su configuración la presencia de valores predidos en las variables numéricas.